In [1]:
import tensorflow as tf
import keras
#from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Model
from keras.layers import *
import os
import matplotlib.pyplot as plt
import sys
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/data_small/'
train_dir = base_dir + 'train'
validation_dir = base_dir + 'validation'
test_dir = base_dir + 'test'

## Define model

In [42]:
f1, f2, f3 = 8,16,32
k_size = (3,3)

pics_input = Input(shape = (250, 75, 3))
pics_c1 = Conv2D(filters = f1, kernel_size = k_size, activation = "relu")(pics_input)
pics_m1 = MaxPooling2D((2,2))(pics_c1)
pics_c2 = Conv2D(filters = f2, kernel_size = k_size, activation = "relu")(pics_m1)
pics_m2 = MaxPooling2D((2,2))(pics_c2)
pics_c3 = Conv2D(filters = f3, kernel_size = k_size, activation = "relu")(pics_m2)
pics_m3 = MaxPooling2D((2,2))(pics_c3)
pics_f = Flatten()(pics_m3)
pics_d = Dense(512, activation = "softmax")(pics_f)

nums_input = Input(shape = ([1]))
nums_features = Dense(1, activation = "softmax")(nums_input)

conc_layer = concatenate([pics_d, nums_features])
dense_1 = Dense(10, activation = "softmax")(conc_layer)

model = Model(inputs = [pics_input, nums_input], outputs = dense_1)

In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           (None, 250, 75, 3)   0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 248, 73, 8)   224         input_39[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 124, 36, 8)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 122, 34, 16)  1168        max_pooling2d_6[0][0]            
__________________________________________________________________________________________________
max_poolin